# DATA REPORT - Categorical Models

I choose the allbus2016 dataset to explore the distribution of political attitude in Germany based on various variables. Goals might be a comparision with the German elections 2017 or a check of common prejudices (for example East Germany vs. West Germany, Man vs. Woman etc.).

# Import

The dataset contains hundreds of different variables. This would be a huge model with millions of parameters. Obviously too many to calculate a useful model. Therefore I choose few variables which are important in my eyes. They are:
 - education ('educ')
 - income ('di01a')
 - information whether the person lives in east or west Germany ('eastwest')
 - sex ('sex')
 - age ('age')
 - political interest ('pa02a')
 - preferred party ('pv01')
 
More information about single variables can be gathered in the codebook in the allbus2016 folder.

Let us have a look at the data:

In [1]:
import pandas as pd
data_table = pd.read_stata('data_sets/allbus2016/allbus2016.dta',
                           columns=['educ', 'di01a', 'eastwest', 'sex', 'age', 'pa02a', 'pv01'])
data_table.head()

,educ,di01a,eastwest,sex,age,pa02a,pv01
0,MITTLERE REIFE,1800,NEUE BUNDESLAENDER,FRAU,47,WENIG,DIE LINKE
1,MITTLERE REIFE,2000,NEUE BUNDESLAENDER,MANN,52,STARK,FDP
2,"VOLKS-,HAUPTSCHULE",2500,ALTE BUNDESLAENDER,MANN,61,MITTEL,CDU-CSU
3,"VOLKS-,HAUPTSCHULE",860,ALTE BUNDESLAENDER,FRAU,54,SEHR STARK,DIE LINKE
4,HOCHSCHULREIFE,VERWEIGERT,ALTE BUNDESLAENDER,MANN,71,SEHR STARK,KEINE ANGABE


In [2]:
len(data_table)

3490

# Data cleansing and exploration
In the coming section we want to clean the data in that way that we can understand them easier and use them for fitting a categorical model.

What is bothering us with the current data? 
- The names of the variables/columns are not self-explaining
- Some of the variables/columns have too many possible values (too many categories)
- Some of the variables/columns are not categorical at all

I wrote a script called "data_cleaning_script.py" that obviously cleans the data. 
There, I give more accurate names to the columns, 
delete rows which have unconvinient categoricals (for example: delete rows where the column pv01 (preferred party) says "Datenfehler", "Keine Angabe" and so on), 
transform the continous variables to categoricals (for example: younger than 45 is "JUNG" and older than 44 is "ALT" and so on) 
and I reduce the categories of some categorical variables (for example: only two options for column pa02a (political interest). 

I also saved the result as a .csv file in folder /data_sets.

Let us have a look at the result:

In [3]:
from utils.utils import *
cleaned_data = read_csv_data('data_sets/', 'dataset.csv')
cleaned_data.head()

,a_education,b_income,c_eastwest,d_sex,e_age,f_political_interest,g_preferred_party
0,EHER GEBILDET,EHER NIEDRIG,NEUE BUNDESLAENDER,FRAU,ALT,NEIN,DIE LINKE
1,EHER GEBILDET,EHER NIEDRIG,NEUE BUNDESLAENDER,MANN,ALT,JA,FDP
2,EHER UNGEBILDET,EHER HOCH,ALTE BUNDESLAENDER,MANN,ALT,JA,CDU-CSU
3,EHER UNGEBILDET,EHER NIEDRIG,ALTE BUNDESLAENDER,FRAU,ALT,JA,DIE LINKE
4,EHER GEBILDET,EHER HOCH,ALTE BUNDESLAENDER,MANN,ALT,JA,CDU-CSU


I have also written the function "read_csv_data" which only uses the panda function to imports a .csv file, but also sets the first column as the index and transform each column to type "categorical". There are more useful functions in the utils/utils.py file such as "import_model" and "data_compl". Let us check the length of our cleaned data set:

In [4]:
len(cleaned_data)

2236

Obviously we reduced our date from 3490 to 2251 rows. Let us check the categories of the single variables:

In [5]:
for col in cleaned_data.columns:
    print(cleaned_data[col].values.categories)

Index(['EHER GEBILDET', 'EHER UNGEBILDET'], dtype='object')
Index(['EHER HOCH', 'EHER NIEDRIG'], dtype='object')
Index(['ALTE BUNDESLAENDER', 'NEUE BUNDESLAENDER'], dtype='object')
Index(['FRAU', 'MANN'], dtype='object')
Index(['ALT', 'JUNG'], dtype='object')
Index(['JA', 'NEIN'], dtype='object')
Index(['AFD', 'CDU-CSU', 'DIE GRUENEN', 'DIE LINKE', 'FDP', 'SPD',
       'WUERDE NICHT WAEHLEN'],
      dtype='object')


Also let us check the complexity of a model that we would learn on this data:

In [6]:
compl = 1
for col in cleaned_data.columns:
    c = len(cleaned_data[col].values.categories)
    compl = compl * c
compl

448

So this means we would get a model with 448 parameters. That is a little too many for our data, but fine for a first approach in learning a model on our data!

# Modelling

After we have imported and cleaned our data we would like to fit a categorical model to it. Therefore we import the nececcary libraries from my written code and create an object from the class "Categ" which stands for Categorical Model, followed by saving the cleaned .csv file which we created in the last section:

In [7]:
from modules.models import *
catmod = Categ("allbus2016")
catmod.data = read_csv_data(catmod.path_data, 'dataset.csv')
catmod.data.head()

,a_education,b_income,c_eastwest,d_sex,e_age,f_political_interest,g_preferred_party
0,EHER GEBILDET,EHER NIEDRIG,NEUE BUNDESLAENDER,FRAU,ALT,NEIN,DIE LINKE
1,EHER GEBILDET,EHER NIEDRIG,NEUE BUNDESLAENDER,MANN,ALT,JA,FDP
2,EHER UNGEBILDET,EHER HOCH,ALTE BUNDESLAENDER,MANN,ALT,JA,CDU-CSU
3,EHER UNGEBILDET,EHER NIEDRIG,ALTE BUNDESLAENDER,FRAU,ALT,JA,DIE LINKE
4,EHER GEBILDET,EHER HOCH,ALTE BUNDESLAENDER,MANN,ALT,JA,CDU-CSU


We receive the same output as above. Next thing to do is fitting a categorical model on our data. This can be done with the implemented function "selection". It is important that the class variable "data" of our class object "catmod" is assigned to the data.

In [8]:
catmod.model = catmod.selection(save=True)

Saved model in:  models/


In [9]:
catmod.model.head()

,a_education,b_income,c_eastwest,d_sex,e_age,f_political_interest,g_preferred_party,p
0,EHER GEBILDET,EHER HOCH,ALTE BUNDESLAENDER,FRAU,ALT,JA,AFD,0.000894454
1,EHER GEBILDET,EHER HOCH,ALTE BUNDESLAENDER,FRAU,ALT,JA,CDU-CSU,0.00536673
2,EHER GEBILDET,EHER HOCH,ALTE BUNDESLAENDER,FRAU,ALT,JA,DIE GRUENEN,0.00268336
3,EHER GEBILDET,EHER HOCH,ALTE BUNDESLAENDER,FRAU,ALT,JA,DIE LINKE,0
4,EHER GEBILDET,EHER HOCH,ALTE BUNDESLAENDER,FRAU,ALT,JA,FDP,0.00268336


What happened? So we created a model that is in this case basically a table with the same columns as in the data plus an extra column "p" which stands for probabilty of the vector. After that we counted the occurence of each possible vector in the data and divided it in the end by the total number of rows in the data. The result is a "score" which shows how many times a specific combination of the variables occurs in the data. What can we do now? Well, we could calculate the maximum of the probability density. In other words the combination of variables which occurs the most:

In [10]:
max = catmod.argmaximum()

OUTPUT############################
Score:  0.028622540250447227  - Prob:  2.862254025044723 %
Index/Indizes of Sol:  [114]
MAX at: 
[a_education                  EHER GEBILDET
b_income                      EHER NIEDRIG
c_eastwest              ALTE BUNDESLAENDER
d_sex                                 FRAU
e_age                                  ALT
f_political_interest                    JA
g_preferred_party              DIE GRUENEN
Name: 114, dtype: object]
OUTPUT - END #####################


So if we ask for a random person in Germany, the most likely person would be a high educated, old (older than 44) woman from West Germany with a low income, political interested and the green party as political preference. 

What else can we do? Well, we can check if the argmaximum is really the vector with this score.

In [11]:
catmod.dens(max[0])

0.028622540250447227

Yup, it is. Btw: Why do I call the vector with "max[0]"?. Well, it could happen that there are more than one maximum. In this case we get a list (max[0], max[1] ..) of vectors which have the highest score.

# Prediction

In this section I want to use my model in its functions (Marginalization, Conditionalization, Aggregation) to give some possible answers on some hypotheses I thought about.

Hypotheses:
- It is more likely to vote for "AFD" in East Germany than in West Germany when you are an old man (1)
- It is most likely to vote for "DIE GRUENEN", when you are a young, high educated woman (2)
- It is most likely to have a high income, when you are high educated and have political interest (3)


In [12]:
catmod.model.head()

,a_education,b_income,c_eastwest,d_sex,e_age,f_political_interest,g_preferred_party,p
0,EHER GEBILDET,EHER HOCH,ALTE BUNDESLAENDER,FRAU,ALT,JA,AFD,0.000894454
1,EHER GEBILDET,EHER HOCH,ALTE BUNDESLAENDER,FRAU,ALT,JA,CDU-CSU,0.00536673
2,EHER GEBILDET,EHER HOCH,ALTE BUNDESLAENDER,FRAU,ALT,JA,DIE GRUENEN,0.00268336
3,EHER GEBILDET,EHER HOCH,ALTE BUNDESLAENDER,FRAU,ALT,JA,DIE LINKE,0
4,EHER GEBILDET,EHER HOCH,ALTE BUNDESLAENDER,FRAU,ALT,JA,FDP,0.00268336


In [13]:
pred1 = catmod.marg_list(["a_education", "b_income", "f_political_interest"])
pred1 = catmod.cond("d_sex", "MANN", pred1)
pred1 = catmod.cond("e_age", "ALT", pred1)
max0 = catmod.cond("g_preferred_party", "AFD", pred1)
max0

,c_eastwest,d_sex,e_age,g_preferred_party,p
0,ALTE BUNDESLAENDER,MANN,ALT,AFD,0.517241
1,NEUE BUNDESLAENDER,MANN,ALT,AFD,0.482759


What do we get: If you choose randomly an old men who votes for AFD, it's more likely he is from West Germany. Simply because there live more people. What do we actually want to know is: If we choose someone from East and West Germany, where is it more likely to vote for AFD? 

In [14]:
max1 = catmod.cond("c_eastwest", "NEUE BUNDESLAENDER", pred1)
max2 = catmod.cond("c_eastwest", "ALTE BUNDESLAENDER", pred1)
maxima = []
maxima.append(catmod.dens(["NEUE BUNDESLAENDER", "MANN", "ALT", "AFD"], max1))
maxima.append(catmod.dens(["ALTE BUNDESLAENDER", "MANN", "ALT", "AFD"], max2))
maxima

[0.18983050847457622, 0.12903225806451613]

Now we get what we have expected. The probabilty to vote for "AFD" is higher when you are in East Germany (~19,0%) than in West Germany (~12,9%). 

In [15]:
pred2 = catmod.marg_list(["b_income", "c_eastwest", "f_political_interest"])
pred2 = catmod.cond("a_education", "EHER GEBILDET", pred2)
pred2 = catmod.cond("d_sex", "FRAU", pred2)
pred2 = catmod.cond("e_age", "JUNG", pred2)
pred2

,a_education,d_sex,e_age,g_preferred_party,p
0,EHER GEBILDET,FRAU,JUNG,AFD,0.064067
1,EHER GEBILDET,FRAU,JUNG,CDU-CSU,0.295265
2,EHER GEBILDET,FRAU,JUNG,DIE GRUENEN,0.239554
3,EHER GEBILDET,FRAU,JUNG,DIE LINKE,0.089136
4,EHER GEBILDET,FRAU,JUNG,FDP,0.041783
5,EHER GEBILDET,FRAU,JUNG,SPD,0.200557
6,EHER GEBILDET,FRAU,JUNG,WUERDE NICHT WAEHLEN,0.069638


In [16]:
max = catmod.argmaximum(pred2)

OUTPUT############################
Score:  0.295264623955  - Prob:  29.526462395543174 %
Index/Indizes of Sol:  [1]
MAX at: 
[a_education          EHER GEBILDET
d_sex                         FRAU
e_age                         JUNG
g_preferred_party          CDU-CSU
Name: 1, dtype: object]
OUTPUT - END #####################


This hypthesis is wrong, but if we consider that the green party is a quite small party, we should compare it only with other small parties:

In [17]:
pred2 = catmod.cond("g_preferred_party", ["AFD", "DIE GRUENEN", "SPD", "DIE LINKE", "FDP", "NPD"], pred2)
max = catmod.argmaximum(pred2)

OUTPUT############################
Score:  0.377192982456  - Prob:  37.719298245614034 %
Index/Indizes of Sol:  [1]
MAX at: 
[a_education          EHER GEBILDET
d_sex                         FRAU
e_age                         JUNG
g_preferred_party      DIE GRUENEN
Name: 1, dtype: object]
OUTPUT - END #####################


Now we get what we wanted.

In [18]:
pred3 = catmod.marg_list([2, 3, 4, 6])
pred3 = catmod.cond("a_education", "EHER GEBILDET", pred3)
#pred3 = catmod.cond("b_income", "EHER HOCH", pred3)
pred3 = catmod.cond("f_political_interest", "JA", pred3)
#max = catmod.argmaximum(pred3)
pred3

,a_education,b_income,f_political_interest,p
0,EHER GEBILDET,EHER HOCH,JA,0.227894
1,EHER GEBILDET,EHER NIEDRIG,JA,0.772106


This hypotheses seems to be wrong, but if you consider that we have the border between rich and not rich at "2500" and that you already have a high education with "Mittlere Reife", then it might be comprehensibly. Now I ask a few other couple of questions.

Questions:
- What is the most likely preferred party, when you are political interested? (4)
- What is the most likely sex, when you are high educated? (5)
- What is the most likely sex, when you have a high income? (6)
- What is the most likely part in Germany you live (East Germany / West Germany), when your preferred party is "AFD" or "DIE LINKE"? (7)

In [19]:
pred4 = catmod.marg_list([0, 1, 2, 3, 4])
pred4 = catmod.cond("f_political_interest", "JA", pred4)
max = catmod.argmaximum(pred4)

OUTPUT############################
Score:  0.283464566929  - Prob:  28.346456692913392 %
Index/Indizes of Sol:  [1]
MAX at: 
[f_political_interest         JA
g_preferred_party       CDU-CSU
Name: 1, dtype: object]
OUTPUT - END #####################


We also can check the score of the other parties in the model:

In [20]:
pred4

,f_political_interest,g_preferred_party,p
0,JA,AFD,0.114173
1,JA,CDU-CSU,0.283465
2,JA,DIE GRUENEN,0.163667
3,JA,DIE LINKE,0.104049
4,JA,FDP,0.079303
5,JA,SPD,0.196288
6,JA,WUERDE NICHT WAEHLEN,0.059055


In [21]:
pred5 = catmod.marg_list([1, 2, 4, 5, 6])
pred5 = catmod.cond("a_education", "EHER GEBILDET", pred5)
max = catmod.argmaximum(pred5)

OUTPUT############################
Score:  0.508761329305  - Prob:  50.8761329305136 %
Index/Indizes of Sol:  [0]
MAX at: 
[a_education    EHER GEBILDET
d_sex                   FRAU
Name: 0, dtype: object]
OUTPUT - END #####################


Well, women are already better in school, so it is not surprising they have a better education in the end, even though it is not a big difference.

In [22]:
pred6 = catmod.marg_list([0, 2, 4, 5, 6])
pred6 = catmod.cond("b_income", "EHER HOCH", pred6)
max = catmod.argmaximum(pred6)

OUTPUT############################
Score:  0.768041237113  - Prob:  76.80412371134021 %
Index/Indizes of Sol:  [1]
MAX at: 
[b_income    EHER HOCH
d_sex            MANN
Name: 1, dtype: object]
OUTPUT - END #####################


This is not surprising as well, if you consider that women take care of the children and are more likely to have half-time jobs etc.

In [23]:
pred7 = catmod.marg_list([0, 1, 3, 4, 5])
max0 = catmod.cond("g_preferred_party", ["AFD", "DIE LINKE"], pred7)
max0 = catmod.marg("g_preferred_party", max0)
max = catmod.argmaximum(max0)

OUTPUT############################
Score:  0.543046357616  - Prob:  54.30463576158942 %
Index/Indizes of Sol:  [1]
MAX at: 
[c_eastwest    NEUE BUNDESLAENDER
Name: 1, dtype: object]
OUTPUT - END #####################


This is not very surprising as well, since the people in East Germany do not have a strong connection to the older parties like CDU-CSU / SPD / FDP, because they were voting since the Wende. But here again like in the first case: It refers to a totally random person in Germany who votes for the AFD or DIE LINKE, it's more likely that this person is from East Germany. Let's check this more accurate:

In [24]:
max1 = catmod.cond("c_eastwest", "NEUE BUNDESLAENDER", pred7)
max2 = catmod.cond("c_eastwest", "ALTE BUNDESLAENDER", pred7)
max1

,c_eastwest,g_preferred_party,p
0,NEUE BUNDESLAENDER,AFD,0.152833
1,NEUE BUNDESLAENDER,CDU-CSU,0.230567
2,NEUE BUNDESLAENDER,DIE GRUENEN,0.100132
3,NEUE BUNDESLAENDER,DIE LINKE,0.171278
4,NEUE BUNDESLAENDER,FDP,0.047431
5,NEUE BUNDESLAENDER,SPD,0.173913
6,NEUE BUNDESLAENDER,WUERDE NICHT WAEHLEN,0.123847


So in East Germany we have a relative proportion of ~0.15 + ~0.17 = 0.32 = 32% of AFD and DIE LINKE.

In [25]:
max2

,c_eastwest,g_preferred_party,p
0,ALTE BUNDESLAENDER,AFD,0.083277
1,ALTE BUNDESLAENDER,CDU-CSU,0.293162
2,ALTE BUNDESLAENDER,DIE GRUENEN,0.180095
3,ALTE BUNDESLAENDER,DIE LINKE,0.056872
4,ALTE BUNDESLAENDER,FDP,0.088693
5,ALTE BUNDESLAENDER,SPD,0.218687
6,ALTE BUNDESLAENDER,WUERDE NICHT WAEHLEN,0.079215


In contrast to West Germany, here we have a relative proportion of ~0.08 + 0.05 = 0.13 = 13% of AFD and DIE LINKE.

# Sampling

Now we want some sample points from our fitted model. This can be done we our implemented function:

In [26]:
sampling300 = catmod.sampling(300, save=True)

Saved samples in:  samples/


Or with even more sample points, but I have already done it, so I just read the .csv file:

In [27]:
#sampling300 = read_csv_data(catmod.path_samples, "sampling_300_allbus2016.csv")
sampling5000 = read_csv_data(catmod.path_samples, "sampling_5000_allbus2016.csv")

At last, we can verify our model and our samples by comparing them through the mean absolute / square error. At first we have to fit a categorical model on the samples, then we can compare the models:

In [28]:
model300 = catmod.selection(save=False, data=sampling300)
model5000 = catmod.selection(save=False, data=sampling5000)

In [29]:
catmod.error(model300, method="abs")

0.6631007751937971

In [30]:
catmod.error(model5000, method="abs")

0.17290590339892653

This shows how it should be: The mean absolute error decreases when we increase the number of sampling points.

# Conclusion

In the end I have the impression that our model approximates the given data very well, even though we have too many parameters (448) for too less data (2236). This means the model is probably a bit overfitted. We also could improve the model with laplace smoothing or similar methods, but for this application the approximation is fine.